# Разработка системы распознавания предметов интерьера в потоковом видео

## Часть 2: Выявление частоты классов в датасете

In [1]:
%matplotlib inline

#System
import os
import shutil
import random

#Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from skimage.io import imread

#Seed
seed = 99
np.random.seed(seed)
random.seed(seed)

In [2]:
from scripts_.PathFinder import PathFinder
from scripts_.SegEncoder import SegEncoder
from scripts_.ClassList import ClassList

pf = PathFinder()
se = SegEncoder("static/objectnames.txt")

img_dir = 'data/ADE20K_filtred/images/'

### 1. Класс для работы с данными

Напишем класс для работы с данными

Выделим все объекты из папки, сортируем их по количествую вхождений, установив порог для минимального числа вхождений

In [3]:
%%time
cl = ClassList(img_dir, min_obj=50, progress_step=500)

Size: 7166
Done: 500
Done: 1000
Done: 1500
Done: 2000
Done: 2500
Done: 3000
Done: 3500
Done: 4000
Done: 4500
Done: 5000
Done: 5500
Done: 6000
Done: 6500
Done: 7000
CPU times: user 3min 34s, sys: 35.9 s, total: 4min 10s
Wall time: 5min 40s


In [4]:
cl.class_list[:10]

[['-'],
 ['wall'],
 ['chair'],
 ['floor, flooring'],
 ['painting, picture'],
 ['cabinet'],
 ['windowpane, window'],
 ['table'],
 ['light, light source'],
 ['ceiling']]

In [5]:
cl.size()

193

### 2. Удаление лишних классов

In [6]:
cl.remove_class('curtain, drape, drapery, mantle, pall' )
cl.remove_class('plant, flora, plant life')
cl.remove_class('bottle')
cl.remove_class('vase')
cl.remove_class('box')
cl.remove_class('wall socket, wall plug, electric outlet, electrical outlet, outlet, electric receptacle')
cl.remove_class('flower')
cl.remove_class('plate')
cl.remove_class('glass, drinking glass')
cl.remove_class('mirror')
cl.remove_class('pot, flowerpot')
cl.remove_class('towel')
cl.remove_class('bowl')
cl.remove_class('plaything, toy')
cl.remove_class('basket, handbasket')
cl.remove_class('pot')
cl.remove_class('switch, electric switch, electrical switch')
cl.remove_class('column, pillar')
cl.remove_class('figurine, statuette')
cl.remove_class('tray')
cl.remove_class('railing, rail')
cl.remove_class('jar')
cl.remove_class('napkin, table napkin, serviette')
cl.remove_class('fruit')
cl.remove_class('mug')
cl.remove_class('pool ball')
cl.remove_class('shoe')
cl.remove_class('hood, exhaust hood')
cl.remove_class('pool table, billiard table, snooker table')
cl.remove_class('bag')
cl.remove_class('pitcher, ewer')
cl.remove_class('place mat')
cl.remove_class('cup')
cl.remove_class('bucket, pail')
cl.remove_class('knife')
cl.remove_class('toilet tissue, toilet paper, bathroom tissue')
cl.remove_class('ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin')
cl.remove_class('can, tin, tin can')
cl.remove_class('tree')
cl.remove_class('sculpture')
cl.remove_class('signboard, sign')
cl.remove_class('soap dispenser')
cl.remove_class('pen')
cl.remove_class('cue, cue stick, pool cue, pool stick')
cl.remove_class('rod')
cl.remove_class('teacup')
cl.remove_class('food, solid food')
cl.remove_class('pottery, clayware')
cl.remove_class('arcade machine')
cl.remove_class('dishrag, dishcloth')
cl.remove_class('ball')
cl.remove_class('towel rack, towel horse')
cl.remove_class('soap')
cl.remove_class('canister, cannister, tin')
cl.remove_class('bag, handbag, pocketbook, purse')
cl.remove_class('paper towel')
cl.remove_class('sky')
cl.remove_class('fork')
cl.remove_class('pipe, pipage, piping')
cl.remove_class('saucepan')
cl.remove_class('soap dish')
cl.remove_class('ashtray')
cl.remove_class('bag, traveling bag, travelling bag, grip, suitcase')
cl.remove_class('building, edifice')
cl.remove_class('spoon')
cl.remove_class('partition, divider')
cl.remove_class('coffee cup')
cl.remove_class('shutter')
cl.remove_class('backpack, back pack, knapsack, packsack, rucksack, haversack')
cl.remove_class('poster, posting, placard, notice, bill, card')
cl.remove_class('bouquet, corsage, posy, nosegay')
cl.remove_class('microphone, mike')
cl.remove_class('mousepad, mouse mat')
cl.remove_class('machine')
cl.remove_class('remote control, remote')
cl.remove_class('vent, venthole, vent-hole, blowhole')
cl.remove_class('spectacles, specs, eyeglasses, glasses')
cl.remove_class('hanger')
cl.remove_class('countertop')
cl.remove_class('beam')
cl.remove_class('heater, warmer')
cl.remove_class('kettle, boiler')
cl.remove_class('bannister, banister, balustrade, balusters, handrail')
cl.remove_class('toaster')
cl.remove_class('washer, automatic washer, washing machine')
cl.remove_class('piano, pianoforte, forte-piano')
cl.remove_class('printer')
cl.remove_class('dishwasher, dish washer, dishwashing machine')
cl.remove_class('loudspeaker, speaker, speaker unit, loudspeaker system, speaker system')

True

In [7]:
cl.size()

104

### 3. Объединение классов

In [8]:
cl.join('floor, flooring', 'wall')
cl.join('ceiling', 'wall')

cl.join('cushion', 'bed')
cl.join('pillow', 'bed')
cl.join('eiderdown, duvet, continental quilt', 'bed')
cl.join('blanket, cover', 'bed')
cl.join('cradle', 'bed')

cl.join('swivel chair', 'chair')
cl.join('seat', 'chair')
cl.join('armchair', 'chair')
cl.join('stool', 'chair')
cl.join('bench', 'chair')

cl.join('book', 'paper')
cl.join('magazine', 'paper')
cl.join('notebook', 'paper')
cl.join('document, written document, papers', 'paper')
cl.join('booklet, brochure, folder, leaflet, pamphlet', 'paper')
cl.join('ottoman, pouf, pouffe, puff, hassock', 'paper')

cl.join('double door', 'door')
cl.join('doorframe, doorcase', 'door')
cl.join('screen door, screen', 'door')

cl.join('lamp', 'light, light source')
cl.join('sconce', 'light, light source')
cl.join('candlestick, candle holder', 'light, light source')
cl.join('spotlight, spot', 'light, light source')
cl.join('candle, taper, wax light', 'light, light source')
cl.join('fluorescent, fluorescent fixture', 'light, light source')
cl.join('light bulb, lightbulb, bulb, incandescent lamp, electric light, electric-light bulb', 'light, light source')
cl.join('candelabrum, candelabra', 'light, light source')
cl.join('chandelier, pendant, pendent', 'light, light source')

cl.join('work surface', 'table')
cl.join('desk', 'table')
cl.join('coffee table, cocktail table', 'table')
cl.join('kitchen island', 'table')
cl.join('counter', 'table')
cl.join('console table, console', 'table')

cl.join('blind, screen', 'windowpane, window')

cl.join('television receiver, television, television set, tv, tv set, idiot box, boob tube, telly, goggle box', 'monitor, monitoring device')
cl.join('screen, crt screen', 'monitor, monitoring device')
cl.join('screen, silver screen, projection screen', 'monitor, monitoring device')

cl.join('faucet, spigot', 'sink')

cl.join('sofa, couch, lounge', 'bed')

cl.join('bulletin board, notice board', 'blackboard, chalkboard')
cl.join('board, plank', 'blackboard, chalkboard')

cl.join('skylight, fanlight', 'windowpane, window')
cl.join('pane, pane of glass, window glass', 'windowpane, window')

cl.join('laptop, laptop computer', 'computer, computing machine, computing device, data processor, electronic computer, information processing system')
cl.join('keyboard', 'computer, computing machine, computing device, data processor, electronic computer, information processing system')
cl.join('system', 'computer, computing machine, computing device, data processor, electronic computer, information processing system')

cl.join('jacket', 'shirt')
cl.join('sweater, jumper', 'shirt')
cl.join('trouser, pant', 'shirt')
cl.join('hat, chapeau, lid', 'shirt')
cl.join('shirt', 'apparel, wearing apparel, dress, clothes')

cl.join('grill, grille, grillwork', 'stove, kitchen stove, range, kitchen range, cooking stove')

cl.join('microwave, microwave oven', 'oven')

cl.join('shelf', 'cabinet')
cl.join('bookcase', 'cabinet')
cl.join('wardrobe, closet, press', 'cabinet')
cl.join('rack', 'cabinet')

cl.join('stairway, staircase', 'stairs, steps')
cl.join('step, stair', 'stairs, steps')

cl.join('tapestry, tapis', 'rug, carpet, carpeting')

cl.join('drawing', 'painting, picture')

cl.join('chest', 'chest of drawers, chest, bureau, dresser')
cl.join('drawer', 'chest of drawers, chest, bureau, dresser')

cl.join('buffet, counter, sideboard', 'bar')
cl.join('booth, cubicle, stall, kiosk', 'bar')

cl.join('shower stall, shower bath', 'shower')

cl.join('teapot', 'coffee maker')

cl.join('radiator', 'air conditioner, air conditioning')
cl.join('fan', 'air conditioner, air conditioning')

True

In [9]:
cl.size()

32

In [10]:
cl.class_list

[['-'],
 ['wall', 'floor, flooring', 'ceiling'],
 ['chair', 'swivel chair', 'seat', 'armchair', 'stool', 'bench'],
 ['painting, picture', 'drawing'],
 ['cabinet', 'shelf', 'bookcase', 'wardrobe, closet, press', 'rack'],
 ['windowpane, window',
  'blind, screen',
  'skylight, fanlight',
  'pane, pane of glass, window glass'],
 ['table',
  'work surface',
  'desk',
  'coffee table, cocktail table',
  'kitchen island',
  'counter',
  'console table, console'],
 ['light, light source',
  'lamp',
  'sconce',
  'candlestick, candle holder',
  'spotlight, spot',
  'candle, taper, wax light',
  'fluorescent, fluorescent fixture',
  'light bulb, lightbulb, bulb, incandescent lamp, electric light, electric-light bulb',
  'candelabrum, candelabra',
  'chandelier, pendant, pendent'],
 ['door', 'double door', 'doorframe, doorcase', 'screen door, screen'],
 ['bed',
  'cushion',
  'pillow',
  'eiderdown, duvet, continental quilt',
  'blanket, cover',
  'cradle',
  'sofa, couch, lounge'],
 ['person, i

In [11]:
cl.save_class_list('static/class_list.txt')

### 4. Совмещение с индексами классов

Запишем правила перекодирования классов в файл

In [12]:
cl.save_class_encode('static/class_encode.txt')